In [1]:
from os import listdir

### Index creation

In [2]:
%%writefile indexBuilding.R
library(Rsubread)
setwd("/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/03_readsToDeRepContigs")
Inputs<-commandArgs(trailingOnly=TRUE)

#Build index
buildindex(basename="referenceIndex",reference=Inputs[1])#allContigs.Derep.fna

Overwriting indexBuilding.R


In [3]:
%%bash
Rscript indexBuilding.R /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/01_Derep/allContigs.DeRep.fna


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
       Rsubread 1.28.0

//=========================== indexBuilder setting ===========================\\
||                                                                            ||
||                Index name : referenceIndex                                 ||
||               Index space : base-space                                     ||
||                    Memory : 8000 Mbytes                                    ||
||          Repeat threshold : 100 repeats                                    ||
||  Distance to next subread : 3                    

### Mapping

In [4]:
%%writefile mapReads.Rsubread.R

library(Rsubread)
setwd("/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/03_readsToDeRepContigs")
Inputs<-commandArgs(trailingOnly=TRUE)
#Reads Mapping
#Inputs[1] 8L_PF_scythed_tagcleaned_sickled_Ab1_paired_R1.fastq
#Inputs[2] 8L_PF_scythed_tagcleaned_sickled_Ab1_paired_R2.fastq
align(index="referenceIndex",readfile1=Inputs[1],readfile2=Inputs[2],output_file=Inputs[3],minFragLength=425,maxFragLength=875,phredOffset=33,nthreads=6,input_format="FASTQ",output_format="BAM")#Ab1.bam

Overwriting mapReads.Rsubread.R


In [5]:
fw = open("runMapreads.list.txt","w")
folder = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp/0_readsVirome/"
for sample in listdir(folder):
    if "R1" in sample:
        print >> fw,"Rscript mapReads.Rsubread.R "+folder+sample+" "+folder+sample.replace("R1","R2")+" "+sample.split('_')[5]+".bam"
fw.close()

In [6]:
%%writefile callRsubread.sh
cat runMapreads.list.txt | while read -r line
do 
    qsub -v FOO="$line" runRsubread.sh
done

Overwriting callRsubread.sh


In [7]:
%%writefile runRsubread.sh
#!/bin/bash
#$ -N subread
#$ -l h_vmem=4G
#$ -pe parallel 6
#$ -l h_rt=06:00:00
#$ -m ea
#$ -M jmoreno@tuebingen.mpg.de
#$ -cwd

export PATH=/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/anaconda3/bin:$PATH
source activate py2_leo
line=$FOO
eval $line

Overwriting runRsubread.sh


In [8]:
%%bash 
chmod +x callRsubread.sh
./callRsubread.sh

Your job 1697144 ("subread") has been submitted
Your job 1697145 ("subread") has been submitted
Your job 1697146 ("subread") has been submitted
Your job 1697147 ("subread") has been submitted
Your job 1697148 ("subread") has been submitted
Your job 1697149 ("subread") has been submitted
Your job 1697150 ("subread") has been submitted
Your job 1697151 ("subread") has been submitted
Your job 1697152 ("subread") has been submitted
Your job 1697153 ("subread") has been submitted
Your job 1697154 ("subread") has been submitted
Your job 1697155 ("subread") has been submitted
Your job 1697156 ("subread") has been submitted
Your job 1697157 ("subread") has been submitted
Your job 1697158 ("subread") has been submitted
Your job 1697159 ("subread") has been submitted
Your job 1697160 ("subread") has been submitted
Your job 1697161 ("subread") has been submitted
Your job 1697162 ("subread") has been submitted
Your job 1697163 ("subread") has been submitted
Your job 1697164 ("subread") has been su

In [ ]:
%%bash
mkdir subReadLog
mv subread.o* subReadLog/

In [9]:
folder = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/03_readsToDeRepContigs/"
fw = open("sortBam.list.txt",'w')
for bam in listdir(folder):
    if bam[-3:]=="bam":
        sample = bam.split('.')[0]
        output = sample + ".sorted.bam"
        #fw.write("samtools sort -T tmp/"+sample+" -o "+output+" "+bam+"\n")
        fw.write("samtools sort "+folder+bam+" > "+folder+sample+".sorted.bam\n")
fw.close()

In [10]:
%%bash
cat sortBam.list.txt | while read -r line
do
    echo $line | qsub -N sort -l h_vmem=4G -l h_rt=01:00:00 -cwd 
done

Your job 1697190 ("sort") has been submitted
Your job 1697191 ("sort") has been submitted
Your job 1697192 ("sort") has been submitted
Your job 1697193 ("sort") has been submitted
Your job 1697194 ("sort") has been submitted
Your job 1697195 ("sort") has been submitted
Your job 1697196 ("sort") has been submitted
Your job 1697197 ("sort") has been submitted
Your job 1697198 ("sort") has been submitted
Your job 1697199 ("sort") has been submitted
Your job 1697200 ("sort") has been submitted
Your job 1697201 ("sort") has been submitted
Your job 1697202 ("sort") has been submitted
Your job 1697203 ("sort") has been submitted
Your job 1697204 ("sort") has been submitted
Your job 1697205 ("sort") has been submitted
Your job 1697206 ("sort") has been submitted
Your job 1697207 ("sort") has been submitted
Your job 1697208 ("sort") has been submitted
Your job 1697209 ("sort") has been submitted
Your job 1697210 ("sort") has been submitted
Your job 1697211 ("sort") has been submitted
Your job 1

In [11]:
%%bash
WORKDIR=/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/03_readsToDeRepContigs
#mkdir $WORKDIR/0.bamFiles
mv $WORKDIR/*bam* $WORKDIR/0.bamFiles

mv: cannot move '/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/03_readsToDeRepContigs/0.bamFiles' to a subdirectory of itself, '/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/03_readsToDeRepContigs/0.bamFiles/0.bamFiles'


### Bam to Coverage  (files before cleaning). Skipped this time.

In [47]:
fw = open("depth.beforeClean.sh",'w')
for bam in listdir('.'):
    if "sorted" in bam:
        coverage = bam.split('.')[0]+".coverage"
        fw.write("samtools depth "+bam+" > "+coverage+"\n")
fw.close()

In [ ]:
%%bash
chmod +x depth.beforeClean.sh
./depth.beforeClean.sh

### Remove reads from BAM (Cleaning)

In [12]:
fw = open("runFilterBam.list.txt","w")
folder = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/03_readsToDeRepContigs/0.bamFiles/"
contaminantReadsFolder = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp/2_readsCleaning/9.readsForDepletion.bacteria/"
for bam in listdir(folder):
    if "sorted" in bam:
        sample = bam.split('.')[0]
        print >> fw , "picard-tools FilterSamReads I="+folder+bam+" O="+folder+sample+".clean.bam READ_LIST_FILE="+contaminantReadsFolder+sample+".reads.contaminants.csv FILTER=excludeReadList"
fw.close()

In [13]:
%%bash
cat runFilterBam.list.txt | while read -r line
do
    echo $line | qsub -N filter -l h_vmem=8G -l h_rt=01:00:00 -cwd
done

Your job 1697232 ("filter") has been submitted
Your job 1697233 ("filter") has been submitted
Your job 1697234 ("filter") has been submitted
Your job 1697235 ("filter") has been submitted
Your job 1697236 ("filter") has been submitted
Your job 1697237 ("filter") has been submitted
Your job 1697238 ("filter") has been submitted
Your job 1697239 ("filter") has been submitted
Your job 1697240 ("filter") has been submitted
Your job 1697241 ("filter") has been submitted
Your job 1697242 ("filter") has been submitted
Your job 1697243 ("filter") has been submitted
Your job 1697244 ("filter") has been submitted
Your job 1697245 ("filter") has been submitted
Your job 1697246 ("filter") has been submitted
Your job 1697247 ("filter") has been submitted
Your job 1697248 ("filter") has been submitted
Your job 1697249 ("filter") has been submitted
Your job 1697250 ("filter") has been submitted
Your job 1697251 ("filter") has been submitted
Your job 1697252 ("filter") has been submitted
Your job 1697

### Map to coverage list

In [14]:
fw = open("depth.afterClean.sh",'w')
folder = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/03_readsToDeRepContigs/0.bamFiles/"
for bam in listdir(folder):
    if "clean.bam" in bam:
        coverage = bam.replace(".bam",".coverage")
        fw.write("samtools depth "+folder+bam+" > "+folder+coverage+"\n")
fw.close()

In [15]:
%%bash
cat depth.afterClean.sh | while read -r line
do
    echo $line | qsub -N map2coverage -l h_vmem=4G -l h_rt=01:00:00 -cwd
done

Your job 1697318 ("map2coverage") has been submitted
Your job 1697319 ("map2coverage") has been submitted
Your job 1697320 ("map2coverage") has been submitted
Your job 1697321 ("map2coverage") has been submitted
Your job 1697322 ("map2coverage") has been submitted
Your job 1697323 ("map2coverage") has been submitted
Your job 1697324 ("map2coverage") has been submitted
Your job 1697325 ("map2coverage") has been submitted
Your job 1697326 ("map2coverage") has been submitted
Your job 1697327 ("map2coverage") has been submitted
Your job 1697328 ("map2coverage") has been submitted
Your job 1697329 ("map2coverage") has been submitted
Your job 1697330 ("map2coverage") has been submitted
Your job 1697331 ("map2coverage") has been submitted
Your job 1697332 ("map2coverage") has been submitted
Your job 1697333 ("map2coverage") has been submitted
Your job 1697334 ("map2coverage") has been submitted
Your job 1697335 ("map2coverage") has been submitted
Your job 1697336 ("map2coverage") has been sub

In [16]:
%%bash
mkdir /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/03_readsToDeRepContigs/1.coverageFiles.afterClean
mv /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/03_readsToDeRepContigs/0.bamFiles/*clean.coverage /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/03_readsToDeRepContigs/1.coverageFiles.afterClean

In [57]:
#This was not run this time
%%bash

mkdir beforeClean
mv *.coverage beforeClean

mv bamFiles/ 0.bamFiles
mv beforeClean/ 1.coverageFiles.beforeClean
mv afterClean/ 2.coverageFiles.afterClean